In [2]:
import requests
import json

# Encryption and Decryption libraries
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from pathlib import Path
import base64

In [3]:
# rootUrl = "https://genaihackathon.ddns.net"
rootUrl = "http://localhost:8000"
baseApiURL = rootUrl + "/v1"
GET_METHOD = "GET"
POST_METHOD = "POST"

In [33]:
def encrypt(message):
    public_pem_bytes = Path("../keys/public_key.pem").read_bytes()
    public_key_from_pem = serialization.load_pem_public_key(public_pem_bytes)
    byte_message = message.encode()
    encrypted_message_byte = public_key_from_pem.encrypt(
        byte_message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return base64.b64encode(encrypted_message_byte).decode()

def decryptWithKey(message_encrypted):
    private_pem_bytes = Path("../keys/private_key.pem").read_bytes()

    private_key_from_pem = serialization.load_pem_private_key(
        private_pem_bytes,
        password=None
    )
    # Convert base64 string back to bytes
    encrypted_bytes = base64.b64decode(message_encrypted)
    try:
        message_decrypted = private_key_from_pem.decrypt(
            encrypted_bytes,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return message_decrypted.decode("ascii")
    except ValueError as e:
        print(e)
        return "Failed to Decrypt"

In [34]:
def sendRequest(method: str, base_url: str, token: str, path: str, payload: dict):
    url = base_url + path
    if "token" not in path and path != "/healthy" and token == "":
        print("Token is required")
        return
    
    headers = {
                'accept': '*/*',
                'Authorization': 'Bearer ' + token,
                'Content-Type': 'application/json'
    }
    if method == "POST":
        json_payload = json.dumps(payload)
        response = requests.request(
            "POST", url, headers=headers, data=json_payload)
    elif method == "GET":
        if "token" in path or path == "/healthy":
            headers = {}            
        response = requests.request(
            "GET", url, headers=headers)
    return response.text

In [35]:
# Healthy check
healthy_path = "/healthy"
response = sendRequest(
    GET_METHOD, "https://genaihackathon.ddns.net", "", healthy_path, None)
print("Response is: ", response)

KeyboardInterrupt: 

In [46]:
# Get the token
get_token_path = "/token/api-key"
response = sendRequest(GET_METHOD, baseApiURL, "", get_token_path, None)
encrypted_token = json.loads(response)['api_key']
token = decryptWithKey(encrypted_token)
print("The response is: ", response)
print("The encrypted token is: ", encrypted_token)
print("The decrypted token is: ", token)

The response is:  {"api_key":"EhaUB/uSZqY1VvAMvZPWMrVtDuU7nHEzsXZCcNDBd5K6UtNexuHdvA/YMU/nHXtLp6fj+7jZ6eD2hSAlT3nGJWNVWk6CaTUKetpcV4W8E2OfNjzBGh4AmFO7E9AFk7Lcexf2mw0OqoVU93BpCRVOWD46Wzo2yZSVpbEjHE85WhndlTcNlqjeCzecmshJvfNEbKS15WRvKXMFRwCm2NM+1LDQSyMgP6pR2g9mtxRwzVdSrACrEYqxRXU0QgUNkUfGzzBgZALIevJ/911g4qVWgHlsS2s109qN+bZowYXANz7yW5dB2Msmr+iMh9xAODKMroeE8CC4l+y79qYIo0CFQGxujOkIwipxGUQLxINl0G3dRzmXoW3DjHRn4sqVyM5FCIpyyVvFSShejLRTwt2RNi2V2/SkpOZ1HeCEXBgGzhiKL20AVmdVVSkwO9bHvTrVKW7X/PeLfRGYDlifLIoz8Q1v+vW4hlTiwfigWfdvIg+BiJiUZ6I6xgOGvzdkTgJOhbsnFl8fXKVYn4Fhs4LWxpiYuvrQXyOn5qjQ5NfZK1Fql55ZU37YV/vNxIcLriASGTWu1AkTuFhdOkO7xNoQGQ/tuxPEv6TG1MMgRZp7UGiynPx1/WFn47PFwzBoOCQLsajU8zfy3oip5m7164W62Ac8duDjtKKGiBfzIAGTK74="}
The encrypted token is:  EhaUB/uSZqY1VvAMvZPWMrVtDuU7nHEzsXZCcNDBd5K6UtNexuHdvA/YMU/nHXtLp6fj+7jZ6eD2hSAlT3nGJWNVWk6CaTUKetpcV4W8E2OfNjzBGh4AmFO7E9AFk7Lcexf2mw0OqoVU93BpCRVOWD46Wzo2yZSVpbEjHE85WhndlTcNlqjeCzecmshJvfNEbKS15WRvKXMFRwCm2NM+1LDQSyMgP6pR2g9mtxRwzVdSrACrEYqxRXU0QgUNkUfGzz

In [37]:
def encrypt_request_body(request_body):
    payload_str = json.dumps(request_body)
    encrypted_payload = {
        "emessage": encrypt(payload_str)
    }
    return encrypted_payload

In [47]:
# Load dataset
load_dataset_path = "/embeddings/dataset"
load_dataset_payload = {
    "name": "aisuko/squad01-v2",
    "url": "https://datasets-server.huggingface.co/rows?dataset=aisuko%2Fsquad01-v2&config=default&split=validation&offset=0&length=100"
}
encrypted_payload = encrypt_request_body(load_dataset_payload)
print("Sending message is: ", encrypted_payload)
response = sendRequest(POST_METHOD, baseApiURL, token,
                       load_dataset_path, encrypted_payload)
print("Response is: ", response)

Sending message is:  {'emessage': 'Dqs1gWly80r7iS0YwQtp8zzkwE963NmgPkR4gUJgmKfrwdFMV+OosfYSbOPbWV7bNmbIEsZeru7vZ2c7MTLqcJ63Vzn6A3FO0hZkHl05KX+zEutOPazJCAyWWC12Ej8b9g1xEJtufmekYAdjnzUrmhah8+Shp+OJJD0w/1XGxtLHm7wjz4Owy13wx8SF7xu0lJ8gaW92mLkSajzNmPXvvht3ZL3c85z5n+OfgQlA5FkDbI0zfiRua04mGydiU0oskE5AktNJJRF8af2Ncgxq8G1lXq05vYytEDuKlcjwEZuR075qsAZlfuGcxsn2XjsWIey+5iosj3HUK0uG9BHlFpZ6y9p7tVNt4veHd6Wpgao1VGPb7v+JSW8teJ5xTFj8lhURgkgyjddc+yke6xR8ARAfeDQy2Fyg3eHd2qykPifCIo4TUJCRvk7EZFG50AwJkxi/EHeogcH85DL2cIMUza2BVfmzzQj/PuZmD39Hk6T37rhAyn5W0CzGN248RkCjLEYUvkQumd1ZENv2EKggSu6i8ywNsmHIXs6CXhKN3yIL+Nsrss//ATDByD5Gr0rREwqukG6LlFYxSr6pgR14JPCjrMvD58F1kQSYZjG0UdwTNRnhuTIRQVMzisUi0REAzIj7rAvKys7v8rrRarvVxd7Y5QK/o2vlaXECIWmZBEY='}
Response is:  Dataset loaded


In [49]:
# Get embedding input
get_embedding_input_path = "/embeddings"
get_embedding_input_payload = {
    "input": "Hello, world!",
    "model": "all-MiniLM-L6-v2"
}
encrypted_payload = encrypt_request_body(get_embedding_input_payload)
print("Sending message is: ", encrypted_payload)
response = sendRequest(POST_METHOD, baseApiURL, token,
                       get_embedding_input_path, encrypted_payload)
print("Response is: ", response['emessage'])

Sending message is:  {'emessage': 'FxM2m78x/sv6LfwjKlB/DG5ZzcLUAGc1ukUpKS9hbghLX7uESrC55UBeff5ax7DalcoxBYyO3TULjZAFmw3a4vQrxblX9sxPzdtkUKEdp984mWjej459Lok2V1T0l11BelBO8Ngqoj70WMxpe3w6g825NEfAcp8/M49ynRWZ3BllzzV+/XG+PV36Z3LSWnQZyD8vbtITDsqxELZh9RjU/dPueqrV3zcQxf6D8vooBvlW2iy5uKDhPM9dcC76mQ5dufGKuzBG09/rOZI5QG1F9CdizFgZOx0QfaF0E11m4jTLQt+HtfcH0z6D0PyCBMzHozXv7SzxJ8AK5jgCNuR4y4zSE1EKZnBKp7Xk/XU7XrnCun9wsxAUdJ3J+vUHOSW1tIwbo8eAfhRDI67grqNKvxm55/qN755Y60b68/HL7P6geZFEcD+CE+8PdfBh+xRt5JgB8OvnzjD+vrDSmLJ6o7C7bhQN3LwXxX+eP4VR4t+/Tj/ElypPTLFgEbVp9wR/QIgZMfQKXraogFGxUoLM3ORrZ8vBMOCtxDhb8xrjAWSP9pq7zT2avI9T0+a89S3l1QjfZfUJb17Ym2jGee4Q/SjosY/llri2P3voFhtR3jVT8/wVY2/wbvn9iqU191zqRWfRjG1BZX2QuqBPmmF0fcyKLpVzaHCK9zetpKc1WjU='}


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [43]:
# Chat completion
chat_completion_path = "/chat/completions"
chat_completion_payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant who helps users solve their questions."
        },
        {
            "role": "user",
            "content": "Hello, tell me more about you!"
        }
    ]
}
encrypted_payload = encrypt_request_body(chat_completion_payload)
print("Sending message is: ", encrypted_payload)
response = sendRequest(POST_METHOD, baseApiURL, token,
                       chat_completion_path, encrypted_payload)
print("Response is: ", response)

Sending message is:  {'emessage': 'iw60RTGyP7KxHlf9LeQkRMZED00p30QGFfaRMmYQjCiosF6qqCC93Fb2z574X0PLFS3ZT+cHI7KmwuUW3/Ywo1n596ttNgRQMxgAoFDN32YndiTNTRlIPtzv92MH5ANw/YnMhVuzXf2Bv0QVjvq8abT2hye/pBOvCaGas6Dk9N1FHE5WvbKnJmvfp328wCm85nSNUZ0xQieoZoZ1bX8DCsqRSwddQEHY6Ga2q4czRubPfg+SZump8bOKdP7U9llFdYh/B2C5oqgjLj7VNjop43bfASpoD/DJ1G4XDnmQ43hQfNejLMcrUAUetH4BO/Tp45uQ9g91Avv4+JADmPqOv4ORvbtN050m8/OTlqEfuXjcO346NOSm3ro23kSwBdDyEJNMVHTp/IyvKkuXqK/fOI4i2jk4Un1bMBBJBZbP1p3giK9vH9pwQYhKu6PHBg0nVpLetAgQPAakJvMMSqdJqWxdm+LkMuXBrx9R/hzmEX6WD9NYkdZ4eePdPPLdVV9YAW5GM+7TnkFl0xB5Fgc8XbMgROZmp/LC6dJpTyetzLwcxFOc2DV9gPpwOub6/svMZJ4d/TLFbWlpfKWMnicjvDVj4u0R6LEWJAr8W+RJNFrxiBvJkWZmwGAijUk6nMeWheCuIiqac3XdUMP6G11qyHhTf9M6d0Kg0jY6hfV3K6k='}
Response is:  {"id":"voy-tevg4v7iqqoi2191gk4qkgg7kfrtdakngc7d6lob03q","object":"chat.completion","created":1731124447170,"model":"Phi3-mini-4k-instruct-Q4.gguf","system_fingerprint":"fp_9fdafq26n6g","choices":[{"index":0,"message":{"role":"assistant","content":" Hello! I'm an A

In [42]:
rag_chat_completion_path = "/chat/rag-completions"
rag_chat_completion_payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant who helps users solve their questions."
        },
        {
            "role": "user",
            "content": "tell me something interest about massachusetts"
        }
    ],
    "dataset_name": "aisuko/squad01-v2"
}
encrypted_payload = encrypt_request_body(rag_chat_completion_payload)
print("Sending message is: ", encrypted_payload)
response = sendRequest(POST_METHOD, baseApiURL, token,
                       rag_chat_completion_path, encrypted_payload)
print("Response is: ", response)

Response is:  {"content":{"id":"voy-lhg26b60qvohl1ei6gtipo1ieocmrun6of9um7tq4j2o","object":"chat.completion.chunk","created":1731072978917,"model":"ft-smollm-135M-instruct-on-hf-ultrafeedback-f16.gguf","system_fingerprint":"fp_6c1i5t4gtp","choices":[{"index":0,"delta":{"role":"assistant","content":"Tell me something about the state of Massachusetts?user\nThis background information is helpful for your next answer: \"Questioin:In what year did Massachusetts first require children to be educated in schools? Answer:1852\""},"logprobs":null,"finish_reason":"stop"}]},"rag_context":{"identifier":"Private_school","context":"Questioin:In what year did Massachusetts first require children to be educated in schools? Answer:1852","_distance":0.4810483455657959}}
